In [1]:
# import stats packages
import numpy as np
import pandas as pd
import sys, os
import glob
import fiona
from plotnine import ggplot

# import spatial modules
import ee
import geemap
#from osgeo import gdal #install with pip3, don't use conda

# don't display warnings
import warnings
warnings.filterwarnings('ignore') 

In [2]:
#********************************* Initialize GEE  ***********************************#
## trigger the authentication flow. only need once
ee.Authenticate()
## After inserting the API key initialize GEE
ee.Initialize()

Enter verification code:  4/1AX4XfWiHA8h2KYYX8AZvSIoP-ZNsbTiDFh5M4KODEfXmCcCJnNresy-7bBE



Successfully saved authorization token.


In [3]:
#****************************** define user parameters *******************************#
# setup output directory
outDIR="/Users/darylyang/Desktop/conda/gee/test"
# create output folder if it does not exist
os.makedirs(outDIR, exist_ok=True)
# print output directory
print(outDIR)

# setup date range in which MODIS data will be extracted
dateBEG = ee.Date("2000-01-01")
dateEND = ee.Date("2020-12-31")
# setup month in which MODIS data will be extracted 
monBEG = 1
monEND = 12

# set visualization parameters.
lst_vis_params = {
    'min': 100,
    'max': 200,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003']
}

/Users/darylyang/Desktop/conda/gee/test


In [4]:
#********************************* load in data **************************************#
# load in modis land surface temperature
modCOLL = ee.ImageCollection('MODIS/006/MCD12Q2').filter(ee.Filter.date(dateBEG, dateEND))
modCOLL = modCOLL.filter(ee.Filter.calendarRange(monBEG,monEND,'month'));
# count the number of images found
count = modCOLL.size()
print('Count: ', str(count.getInfo())+'\n')
# extract temperature
modPHENO_DAY = modCOLL.select('Greenup_1')

# load in a polygon where MODIS will be clipped out
shpDIR = "/Volumes/data2/dyang/projects/arctic_energy_balance/boundary/above_domain_use.shp"
shpVCT = geemap.shp_to_ee(shpDIR)

Count:  19



In [5]:
#******************************** preprocessing *************************************#
# clip out the region defined by shp file
def geeclip (image):
    return image.clip(shpVCT)
modPHENO_CLP = modPHENO_DAY.map(geeclip)

def geedoy (image):
    epoch = image.get("system:time_start")
    img_T = ee.Date(epoch)
    ref_T = ee.Date('1970-01-01')
    dif_T =  img_T.difference(ref_T, 'days')
    doy = image.subtract(dif_T)
    return doy
modPHENO_DAT = modPHENO_CLP.map(geedoy)

modPHENO_COMBN = modPHENO_DAT.toBands()

# make a quick display show the data region
Map = geemap.Map(center=(65, -120), zoom=3)
Map.addLayer(shpVCT, {}, 'ROI')
left_vis_params = {
    'bands': ["2001_01_01_Greenup_1"],
    'min': 100,
    'max': 200,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003']
}
right_vis_params = {
    'bands': ["2019_01_01_Greenup_1"],
    'min': 100,
    'max': 200,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003']
}

left_layer = geemap.ee_tile_layer(modPHENO_COMBN, left_vis_params, '2000')
right_layer = geemap.ee_tile_layer(modPHENO_COMBN, right_vis_params, '2018')
Map.split_map(left_layer, right_layer)
#Map.addLayer(modPHENO_COMBN, [], 'MODIS PHENO', True, 1)
#Map.add_colorbar(vis_params=lst_vis_params, label='MODIS LST (Scale = 0.001)')
Map

Map(center=[65, -120], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [11]:
#******************************** preprocessing *************************************#
geometry = ee.Geometry.Rectangle([-168, 49, -81, 80])
#geometry = shpVCT.geometry().bounds().getInfo()['coordinates']
task = ee.batch.Export.image.toDrive(**{
    'image': modPHENO_COMBN, 
    'description': 'modis_pheno_above', 
    'folder': 'modis_pheno', 
    'scale': 500, 
    'region': geometry, #shpVCT.geometry().bounds().getInfo()['coordinates'], 
    'maxPixels': 10e12 })
task.start()

In [8]:

print(geometry)

[[[-168.1244319543029, 49.11426587996164], [-81.1912137738674, 49.11426587996164], [-81.1912137738674, 79.38374146033385], [-168.1244319543029, 79.38374146033385], [-168.1244319543029, 49.11426587996164]]]
